#### Домашнее задание
* Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9) которые бы
  задавались следующим образом:
* x - последовательность цифр
* y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10
* Задача научить модель предсказывать y(i) по x(i)
* Опробовать RNN, LSTM, GRU длинной 10, 25, 50 , 100

In [1]:
import torch
from torch import nn
import re
import random
import tqdm
import time
import numpy as np
from time import sleep

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [4]:
def num(num):
    x = torch.randint(0, 9, (num, ))
    y = torch.zeros(x.shape[0]).int()
    for i, j in enumerate(x):
        if i == 0:
            y[i] = x[i]
        else:
            y[i] = x[i] + x[0]
            if y[i] >= 10:
                y[i] -= 10
    x = x.reshape(1, num)
    y = y.reshape(1, num)            
    return x, y  


In [5]:
num(5)

(tensor([[1, 3, 4, 6, 6]]), tensor([[1, 4, 5, 7, 7]], dtype=torch.int32))

In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self,  rnnClass, dictionary_size, embedding_size = 10, num_hiddens = 128, num_classes = 10):
        super().__init__()
        self.rnnClass = rnnClass
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        if self.rnnClass == nn.LSTM:
            predictions = self.output(state[0][0])
        else:
            predictions = self.output(state[0])

        return predictions


In [10]:
def sample(preds):
    soft = nn.LogSoftmax(dim=1)    
    softmaxed = soft(preds)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax(1)

def generate_num(i):
    x, y = num(i)
    for j in range(i):

        preds = model((x[0][j]).reshape(1, 1))
        
    return sample(preds), y[0][-1]

In [12]:
rnn = nn.RNN
gru = nn.GRU
lstm = nn.LSTM
nets = {'RNN': rnn, 'GRU': gru, 'LSTM': lstm}
d = [10, 25 , 50, 100]

In [16]:
# for k, v in net.items():
#     model = NeuralNetwork(v, 10, 10, 128, 10)

def net_model(k, v, i, model):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    print("_" * 20)
    print("модель:", k, "Длина:", i)
    
    rnn_dict = {}
    x, y = num(i)
    emb = nn.Embedding(10, 10)
    for ep in range(10):
        for h in range(i):
                   

            n = x[0][h].reshape(1, 1)
            j = y[0][h].view(1,)

            start = time.time()
            train_loss = 0.
            train_passed = 0

            model.train() 

            optimizer.zero_grad()
            answers = model(n)  


            loss = criterion(answers, emb(j))
            train_loss += loss.item()


            loss.backward(retain_graph = True)    
            optimizer.step()
            train_passed += 1    

        print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        model.eval()
        gen = generate_num(i)
        l = np.where((gen[0].squeeze() - gen[1].squeeze())==0)    
        print(f' "Точность"', abs((10-abs(gen[0].squeeze() - gen[1].squeeze())).item())*10, '%')
        print((gen[0].squeeze(), gen[1].squeeze()))
        if (gen[0].squeeze() - gen[1].squeeze()).item() == 0:
            return

In [17]:
for k, v in nets.items():
    for i in d:
        model = NeuralNetwork(v, i)
        net_model(k, v, i, model)

____________________
модель: RNN Длина: 10
Epoch 0. Time: 0.001, Train loss: -0.777
 "Точность" 60 %
(tensor(7), tensor(3, dtype=torch.int32))
Epoch 1. Time: 0.001, Train loss: -1.769
 "Точность" 70 %
(tensor(3), tensor(0, dtype=torch.int32))
Epoch 2. Time: 0.001, Train loss: -2.882
 "Точность" 100 %
(tensor(5), tensor(5, dtype=torch.int32))
____________________
модель: RNN Длина: 25
Epoch 0. Time: 0.002, Train loss: -11.518
 "Точность" 100 %
(tensor(0), tensor(0, dtype=torch.int32))
____________________
модель: RNN Длина: 50
Epoch 0. Time: 0.001, Train loss: -0.052
 "Точность" 80 %
(tensor(1), tensor(3, dtype=torch.int32))
Epoch 1. Time: 0.001, Train loss: -0.995
 "Точность" 60 %
(tensor(4), tensor(8, dtype=torch.int32))
Epoch 2. Time: 0.001, Train loss: -2.210
 "Точность" 10 %
(tensor(0), tensor(9, dtype=torch.int32))
Epoch 3. Time: 0.001, Train loss: -4.372
 "Точность" 80 %
(tensor(6), tensor(8, dtype=torch.int32))
Epoch 4. Time: 0.001, Train loss: -7.774
 "Точность" 100 %
(tensor(0

#### Задание 2:
##### применить LSTM для решения лекционного практического задания

In [18]:
import torch
from torch import nn
import re
import random
import tqdm
import time

In [ ]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [21]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600901


In [22]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [23]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [24]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [25]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [26]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [27]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]),
 tensor(23))

In [28]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0].squeeze())
        return predictions

In [34]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))
model

NeuralNetwork(
  (embedding): Embedding(27, 64)
  (hidden): LSTM(64, 128, batch_first=True)
  (output): Linear(in_features=128, out_features=27, bias=True)
)

In [31]:
X.shape

torch.Size([193075, 40])

In [32]:
model(X[0:1])

tensor([-6.5505e-02, -2.6092e-02,  3.8988e-02, -1.0529e-01,  2.8476e-02,
        -2.0989e-02, -9.6306e-02,  1.1347e-01,  1.1606e-01,  1.0811e-01,
         1.5207e-01,  1.2353e-01, -4.6715e-02, -1.0129e-01,  1.2708e-02,
        -1.3236e-01, -9.8945e-02,  1.0930e-01,  1.2650e-01, -1.4413e-01,
        -1.5080e-02,  3.8489e-02,  4.9186e-02,  7.7933e-02, -3.3575e-02,
        -9.5032e-05,  9.1126e-03], grad_fn=<AddBackward0>)

In [33]:
# embedding = nn.Embedding(len(INDEX_TO_CHAR), 15)
# rnn = nn.LSTM(15,128, batch_first=True)

In [ ]:
# rnn = nn.GRU(15,128, batch_first=True)
# o, s = rnn(embedding(X[0:10]))
# o.shape, s[0].shape

(torch.Size([10, 40, 128]), torch.Size([10, 128]))

In [ ]:
# o, s = rnn(embedding(X[0:10]))

In [ ]:
# o.shape, s[0].shape, s[1].shape

(torch.Size([10, 40, 128]), torch.Size([1, 10, 128]), torch.Size([1, 10, 128]))

In [ ]:
model = model.cuda()

In [38]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred) #.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [39]:
generate_text()

happiness thereupon all the evils living|vstp upfqnqnpmvtfmprjycvwmrjsoemgnsfyxqv


In [40]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [41]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in data:
        X_b, y_b = X_b, y_b #.cuda(),.cuda()
        optimizer.zero_grad()
        answers = model(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

Epoch 0. Time: 90.816, Train loss: 2.180
lately been added a german the author of| mow uve hergus ipsuls thas trogh amourb
Epoch 1. Time: 90.212, Train loss: 1.797
mself that he persistently avoids it and| passe encomity of diskind and evilongo 
Epoch 2. Time: 88.347, Train loss: 1.657
intained in europe is perhaps the best e|nvuth folitn to luokes ad outto he badio
Epoch 3. Time: 87.753, Train loss: 1.568
 nature badly endowed and prepared so th|e act an a than platurally our everther 
Epoch 4. Time: 85.444, Train loss: 1.508
of existence justifies itself no termina|ly itself one in result and attentaty it
Epoch 5. Time: 84.961, Train loss: 1.461
r when the occasion arises the fable of |the intruex a condigniziss and perpation
Epoch 6. Time: 86.122, Train loss: 1.425
 about once before a general depression |from and sebbwles mechantoly to parpysin
Epoch 7. Time: 86.091, Train loss: 1.395
e is an innocence in lying which is the |very see a being somethow well kinds or 
Epoch 8. Time: 8